## Pre-Processing of the Opensource Enron Email Corpus

This dataset contains emails between Staff of the Enron corporation, which declared bankrupcy and dissolution in December of 2001. This dataset is the largest collection of opensource emails in respect to internal organizational interaction, and was released as part of a fraud investigation orchestrated by the US Federal Energy Regulatory Commission. The dataset is comprised of over 0.5M emails between around 150 members of Enron staff (mostly senior management), between 1998 and 2002. A substantial number of the emails involve extrernal agents/entities, and also non-identifiable Enron employees. This notebook seeks to highlight the pre-processing needed to make this dataset fully usable, and provide code to facilitate this.

### Start

#### Importing the dataset
Several different versions of this dataset exist, however the data used for this notebook was sourced from work carried out by the Carnegie Mellon University, a link to which can be found [Here](https://www.cs.cmu.edu/~./enron/). The data is in .csv format, and contains the columns 'Date'{2}, 'From'{3}, 'To'{4}, and 'Content'{13}. A number of additional columns are also present that don't contain data, which is why the 'usecols' function has been used. The content column has been defined as string with the 'dtype' function to counter processing problems later on in the script.

In [12]:
import pandas as pd
df = pd.read_csv('enron_05_17_2015_with_labels_v2.csv', usecols=[2,3,4,13], dtype={13:str})

#### Building a count of email recipients
Emails between staff include:
    - Personal communication between two individuals
    - Group messages to numerous individuals
    - Annoucements to large groups of staff, including emails containing over 100 recipients
    
As amount of people included in an email may be useful to analyze, a count is developed of how many recipients are in each email. In the 'To' column, emails sent to more than one person are seperated by comma, which is counted by the function below, and then 1 is added for each line to account for the original recipient.

In [13]:
df['Included_In_Email'] = df.To.str.count(',')
df['Included_In_Email'] = df['Included_In_Email'].apply(lambda x: x+1)

#### Dropping any lines with missing values, and removing announcement emails
Numerous emails within the dataset contain null values in the content column, which the function below removes. Additionally, as my own research required emails to be split into individual lines, emails with over 15 recipients are removed to reduce the amount of emails used for final analysis. After these steps, overall email count drops from 517,401 to 457,401.

In [14]:
df = df.dropna()
df = df[~(df['Included_In_Email'] >=15)]

#### Seperating emails into a line-per-recipient format
As my research was based around staff networks, emails needed to be seperated into a line-per-line format, which the following function carries out. After splitting emails into an individual recipient format, number of emails increases substantially from 457,401 to 852,253. If you receive an error:

ValueError: Length mismatch: Expected axis has 6 elements, new values have 5 elements

Ignore it, this does not effect outcome.

In [15]:
df['To'] = df.To.str.split(',')
df2 = df.set_index(['From', 'Date', 'content', 'Included_In_Email'])['To'].apply(pd.Series).stack()
df2 = df2.reset_index()
df2.columns = ['From','To','Date','content', 'Included_In_Email']

ValueError: Length mismatch: Expected axis has 6 elements, new values have 5 elements

#### Renaming the 'To' column, dropping unneeded column, and changing column order
The last function renamed the 'To' column as '0', and added an unneeded column named 'level_4' - the following code is used to drop this unneeded column, and rename 'To'. Additionally, a command is used to sort the columns into a more logical format. The previous dataset 'df' is also removed at this point to avoid clutter.

In [ ]:
del df2['level_4']
df2 = df2.rename(columns = {0: 'To'})
df2 = df2[['Date','From','To','content','Included_In_Email']]
del df

#### Cleaning the 'From' and 'To' columns
Emails within the above columns are of the format: frozenset({'phillip.allen@enron.com'}). The following functions are used to remove 'frozenset' from both columns, and also to remove unneeded punctuation (such as < or ').

In [ ]:
df2['From'] = df2['From'].map(lambda x: x.lstrip("frozenset"))
df2['To'] = df2['To'].map(lambda x: x.lstrip("frozenset"))
df2['From'] = df2['From'].str.strip("<\>(/){?}[:]*, ")
df2['To'] = df2['To'].str.strip("<\>(/){?}[:]*, ")
df2['From'] = df2['From'].str.replace("'", "")
df2['To'] = df2['To'].str.replace("'", "")
df2['From'] = df2['From'].str.replace('"', "")
df2['To'] = df2['To'].str.replace('"', "")

#### Accounting for users having different emails
One staff member can have up to four different emails, so the following code uses a dictionary to convert emails for the same person to only one variant. This dictionary is based on work previoiusly carried out by Andres Corrada-Emmanuel, originally published at [Enron Email Webpage](https://www.cs.cmu.edu/~enron/). I have added several emails to this file that were previously unaccounted for, so my final dictionary can be found on my Enron GitHub page [Here](https://github.com/Laurie-Bamber/Enron_Corpus).

In [ ]:
email_dict = pd.read_csv('dict_email.csv')    
df2['From'] = df2.From.replace(email_dict.set_index('Old')['New'])
df2['To'] = df2.To.replace(email_dict.set_index('Old')['New'])
del email_dict

#### Removing emails not containing the tag 'enron.com'
The code below removes all email addresses not containing @enron.com, so only correspondence between Enron staff is left in the dataset.

In [ ]:
df2['Enron'] = df2.From.str.count('@enron')
df2['Enron'] = df2['Enron']+df2.To.str.count('@enron')
df2 = df2[df2.Enron != 0]
df2 = df2[df2.Enron != 1]
del df2['Enron']

#### Adding job-roles for remaining staff
Using a dictionary, two new columns are added to the dataset which contain staff job-roles: 'Sender_Role' and 'Receiver_Role'. This dictionary is based off previous work carried out by Youngser Park, a Research Scientist at the Centre for Imaging Science (CIS), and can be found [Here](http://cis.jhu.edu/~parky/Enron/employees). I have built on this work and included several more employees that I could identify through Google and dataset emails, which can be found on my Enron GitHub page [Here](https://github.com/Laurie-Bamber/Enron_Corpus). This raises the number of identifiable staff to 177, however a substantial number of staff without identifiable roles still remain in the dataset.

In [ ]:
import csv
with open('dict_role.csv') as f:
    role_dict = dict(filter(None, csv.reader(f)))
df2['Sender_Role'] = df2['From'].map(role_dict)
df2['Receiver_Role'] = df2['To'].map(role_dict)
df2 = df2[['Date','From','To','Sender_Role','Receiver_Role','content','Included_In_Email']]
del role_dict

#### Cleaning content column
A substantial amount of non-informative text is found in the content column, mostly in the case of Forwarded messages with extensive subject lines. The text of each fresh email, however, will always follow the tag 'subject: ', so the following code removes all text prior to this tag. Forwarded emails can contain several of these tags, so the .rsplit function is usedy to concentrate on only the last instance of 'subject: '. The dash line below was also included to further clean non-informative text.

In [ ]:
df2['content'] = df2['content'].str.rsplit('Subject: ').str[-1] 
df2['content'] = df2['content'].str.rsplit(' --------------------------- ').str[-1] 

## The following code can be used for the purpose of network analysis

#### Condensing multiple emails between two people into one line, and adding weight
The following code is used to reduce all the emails between two individuals into one line, with a weight (number of emails) column added for each interaction. As all emails between two individuals are now condensed, the use of the 'Date' and 'Content' columns is now non-applicable.

In [ ]:
Weighted = df2.groupby(['From', 'To']).count()

#### General cleaning after condensing
Several unwanted columns are created by condensing the records. The following code subsequently removes these columns, leaving only the columns 'From', 'To', and 'Weight'.

In [ ]:
Weighted['Weight'] = Weighted['Date']
Weighted = Weighted.drop(['Date','Sender_Role','Receiver_Role','content','Included_In_Email'], 1)
Weighted.reset_index(inplace=True)

#### Re-adding job-roles to staff
Condensing the records also removed job-roles from staff, which the following code subsequently re-adds.

In [ ]:
with open('dict_role.csv') as f:
    role_dict = dict(filter(None, csv.reader(f)))
Weighted['Sender_Role'] = Weighted['From'].map(role_dict)
del role_dict

#### Dropping email exchanges below a certain frequency
Lastly, the following code removes all emails with a weight below a pre-determined threshhold. A weight of <= 3 was determined as appropriate in the case of this research.

In [ ]:
Weighted2 = Weighted[~(Weighted['Weight'] <=3)]
Weighted2 = Weighted.dropna()